In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
# train set labels
y = pd.read_csv('../input/commonlitreadabilityprize/train.csv')['target'].values
print(y)

In [ ]:
# oof example, 8 models oof predictions stacked
oof = np.random.rand(8, len(y))
print(oof.shape)

In [ ]:
# Functions for generating sum-to-one weights (for brute-force searching)
def _make_weight_combs_rec(weights, nb_assets, req_wt, current):
    if nb_assets <= 0:
        yield np.array(current, dtype = float)
    else:
        # Discard weights that cannot possibly be used
        while weights and weights[0] + weights[-1] * (nb_assets - 1) < req_wt:
            weights = weights[1:]
        while weights and weights[-1] + weights[0] * (nb_assets - 1) > req_wt:
            weights = weights[:-1]
        # Add all possible weights
        for w in weights:
            current.append(w)
            yield from _make_weight_combs_rec(weights, nb_assets - 1, req_wt - w, current)
            current.pop()
            
def make_weight_combs(min_wt, max_wt, step, nb_assets, req_wt):
    weights = range(min_wt, max_wt + 1, step)
    current = []
    yield from _make_weight_combs_rec(weights, nb_assets, req_wt, current)

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

def func(weights):
    oof_blend = np.tensordot(weights, oof, axes = ((0), (0)))
    return - rmse(y, oof_blend)

In [ ]:
from joblib import Parallel, delayed

best_score = np.inf
best_weights = None
def optimise(w):
    global best_score, best_weights
    w /= 100
    score = func(w)
    if score < best_score:
        best_score = score
        best_weights = w
        print('Best Score:', best_score)
        print('Best Weights:', best_weights)

# gap = 10 means weights are chosen from [0, 0.1, 0.2 ,0.3, ..., 0.9, 1]
gap = 10
total = len(list(make_weight_combs(0, 100, gap, oof.shape[0], 100)))
results = Parallel(
    n_jobs = -1, 
    backend = 'threading', 
#     require = 'sharedmem', 
)(delayed(optimise)(w) for w in tqdm(make_weight_combs(0, 100, gap, oof.shape[0], 100), total = total))

In [ ]:
print(best_weights)

In [ ]:
best_oof_blend = np.tensordot(best_weights, oof, axes = ((0), (0)))
print(best_oof_blend.shape)
print(rmse(y, best_oof_blend))